In [ ]:
    # import necessary libraries
    import subprocess
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import itertools
    import numpy as np
    import seaborn as sns
    import glob
    import cv2

    from sklearn import datasets, linear_model
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn import preprocessing
    from sklearn.metrics import accuracy_score
    from scipy import stats
    from scipy.stats import norm
    from PIL import ImageFile, Image
    from matplotlib import image,pyplot
    from PIL import ImageFile, Image
    from sklearn.model_selection import train_test_split
    from scipy.stats import zscore
    from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
# training,test,devel data index extraction from partition file
partition  = pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/metadata/partition.csv")
partition_train_index=partition[(partition["Partition"]=="train")]["Id"].to_numpy()
partition_devel_index=partition[(partition["Partition"]=="devel")]["Id"].to_numpy()
partition_test_index=partition[(partition["Partition"]=="test")]["Id"].to_numpy()
partition_index = partition[(partition["Partition"]=="train") | (partition["Partition"]=="devel")|(partition["Partition"]=="test")]["Id"].to_numpy()

In [ ]:
ecg_signal= pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/ECG/1.csv")
arousal=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/physio-arousal/1.csv")
valence=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/valence/1.csv")

print(max(ecg_signal['ECG']))
print(min(ecg_signal['ECG']))
print(np.mean(ecg_signal['ECG']))

# Valence and arousal plot

In [ ]:
for i in [31]:
        arousal=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/physio-arousal/31.csv")
        valence=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/valence/31.csv")
        plt.scatter(valence['value'],arousal['value'],s=2) 
        print(i)
        plt.xlim(-1,1)
        plt.ylim(-1,1)
        ax=plt.gca()
        circle=plt.Circle((0, 0), 0.2, color='r',fill=False)
        ax.add_artist(circle)
        plt.plot([-1,1],[0,0])
        plt.plot([0,0],[-1,1])
        plt.title('Label transformation for participant 31')
        plt.show()

# Plot of BPM variation for some participants

In [ ]:
col={}
for i in partition_train_index:
    ecg_signal_raw= pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/BPM/{i}.csv")
    col[i]=ecg_signal_raw['BPM']
    #     sns.boxplot(x=ecg_signal_raw["BPM"])
df=pd.DataFrame(col)
boxplot = df.boxplot(column=[i for i in partition_train_index[:20]]) 


# Label tranformation following the procedure done by https://github.com/face-analysis/emonet/blob/master/emonet/data/affecnet.py

In [ ]:
# for i in labels:
#     data=pd.read_csv(f"D:/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")
#     one_hot_encoded_data = pd.get_dummies(data, columns = ['labels'])
from sklearn.preprocessing import OneHotEncoder


import math
label=[]
label.extend(partition_train_index)
label.extend(partition_devel_index)
for i in label:
    valence=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/valence/{i}.csv")
    arousal=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/physio-arousal/{i}.csv")

    # valence['value']=pd.cut(valence['value'], bins=np.linspace(-1,1, 3),labels=[False,True])
    # arousal['value']=pd.cut(arousal['value'], bins=np.linspace(-1,1, 3),labels=[False,True])
    target=pd.DataFrame()
    target['timestamp']=valence['timestamp']
    target['V_value']=valence['value']
    target['A_value']=arousal['value']

    target['label']=0
    
    target['intensity'] = np.sqrt(target['V_value']**2+target['A_value']**2)

    mask_neutral = target['intensity']<0.2
    mask_stressed=(target['A_value']>=0) & (target['V_value']<=0) & (target['intensity']>=0.2)
    mask_nonstressed= ~(mask_neutral | mask_stressed)
    
    target['label'][mask_neutral] = 0

    target['label'][mask_stressed] = 1
     
    target['label'][mask_nonstressed] = 2
    
    target.to_csv(f"D:/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv",index=False)


#     target['labels'].value_counts()


## Class distribution for the participants

In [ ]:
for i in partition_train_index:
    if i<70:
        target=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")
        print(i)
        plt.hist(target['label'])  
        plt.show()

## Data preparation for ML classification tasks

In [ ]:
bio_signals_train=pd.DataFrame()

for i in partition_train_index:
    bio_signals=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/Normalized_biosignals/{i}.csv")
    target=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")

    bio_signals=bio_signals[['subject_id','timestamp','ECG','resp','BPM']]
    bio_signals['labels']=target['label']
    bio_signals_train = pd.concat([bio_signals_train,bio_signals])
    bio_signals_= pd.concat([bio_signals_,bio_signals_train, bio_signals],axis=1)


In [ ]:
bio_signals_validation=pd.DataFrame()
for i in partition_devel_index:
    bio_signals=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/Normalized_biosignals/{i}.csv")
    target=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")
    fau=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/faus/{i}.csv")
    fau.drop(['timestamp','subject_id'],axis=1,inplace=True)
    bio_signals=bio_signals[['ECG','resp','BPM']]
    bio_signals['labels']=target['label']
    bio_signals_validation = pd.concat([bio_signals_validation, bio_signals])

### Logistic Classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

clf = LogisticRegression(max_iter=1000)
clf.fit(x,y)

y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

### SVM Classification

In [ ]:
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto')
clf_svm.fit(bio_signals_train[x.columns],bio_signals_train['labels'] )

y_pred=clf_svm.predict(bio_signals_validation[x.columns])
print(clf_svm.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf_svm,bio_signals_validation[x.columns], bio_signals_validation['labels'])

### Xgboost classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

# cross_val_score(XGBClassifier(), X, y)

clf = XGBClassifier()
clf.fit(x,y)

y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))


In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

# Downsampling with random resample

In [ ]:
Non_stressed = bio_signals_train[bio_signals_train["labels"] == 2]
stressed  = bio_signals_train[bio_signals_train["labels"] == 1]
Neutral  = bio_signals_train[bio_signals_train["labels"] == 0]

print(Non_stressed.shape)
print(stressed.shape)
print(Neutral.shape)

In [ ]:
from sklearn.utils import resample

Non_stressed = resample(Non_stressed,
             replace=True,
             n_samples=len(stressed),
             random_state=42)
Neutral = resample(Neutral,
             replace=True,
             n_samples=len(stressed),
             random_state=42)

print(Non_stressed.shape)

In [ ]:
bio_signals_train = pd.concat([Non_stressed, stressed, Neutral])

In [ ]:
plt.hist(bio_signals_train['labels'])

In [ ]:
bio_signals_validation=pd.DataFrame()
for i in partition_devel_index:
    bio_signals=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/{i}.csv")
    target=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")
    fau=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/faus/{i}.csv")
    fau.drop(['timestamp','subject_id'],axis=1,inplace=True)
    bio_signals=bio_signals[['ECG','resp','BPM']]
#     valence=valence[['value']]
#     bio_signals[fau.columns]=fau[fau.columns]
    bio_signals['labels']=target['label']
    bio_signals_validation = pd.concat([bio_signals_validation, bio_signals])
# bio_signals_validation['BPM'] = zscore(bio_signals_validation['BPM'])

### Logistic classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

clf = LogisticRegression(max_iter=1000)
clf.fit(x,y)
y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

### Svm classifier

In [ ]:
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto')
clf_svm.fit(bio_signals_train[x.columns],bio_signals_train['labels'] )
y_pred=clf_svm.predict(bio_signals_validation[x.columns])

print(clf_svm.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf_svm,bio_signals_validation[x.columns], bio_signals_validation['labels'])

### Xgboost classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

# cross_val_score(XGBClassifier(), X, y)

clf = XGBClassifier()
clf.fit(x,y)

y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

# Upsampling with SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels']

X_resampled, y_resampled = SMOTE().fit_resample(x, y)
print(Counter(y_resampled))
# print(sorted(Counter(y_resampled).items()))

In [ ]:
plt.hist(y_resampled)

### Logistic Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels']

clf = LogisticRegression(max_iter=1000)
clf.fit(X_resampled, y_resampled)

y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

### SVM classifier_upsampling

In [ ]:
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto')
clf_svm.fit(X_resampled, y_resampled)

y_pred=clf_svm.predict(bio_signals_validation[x.columns])

print(clf_svm.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

### Xgboost classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

# cross_val_score(XGBClassifier(), X, y)

clf = XGBClassifier()
clf.fit(X_resampled, y_resampled)

y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

# UMAP projection of features

In [ ]:
import umap
reducer = umap.UMAP()
from sklearn.preprocessing import StandardScaler


In [ ]:
bio_signals_train=pd.DataFrame()

for i in partition_train_index:
    bio_signals=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/{i}.csv")
    target=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")
    bio_signals=bio_signals[['subject_id','timestamp','ECG','resp','BPM']]
#     valence=valence[['value']]
    bio_signals['labels']=target['label']
    bio_signals_train = pd.concat([bio_signals_train,bio_signals])
#     bio_signals_= pd.concat([bio_signals_,bio_signals_train, bio_signals],axis=1)

In [ ]:
embedding = reducer.fit_transform(scaled_data)
embedding.shape

In [ ]:
classes=['Neutral','Stressed','Non-stressed']

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=bio_signals_train.labels)
#     c = bio_signals_train['labels'])
plt.gca().set_aspect('equal', 'datalim')
cbar = plt.colorbar(boundaries=np.arange(4) -0.5 )
cbar.set_ticks(np.arange(3))
# cbar.set_ticklabels(bio_signals_train.labels)
cbar.set_ticklabels(classes)
plt.title('UMAP projection of the biosignals features', fontsize=10);

In [ ]:
bio_signals_train.labels.map({"Neutral": 0, "Stressed": 1, "Non_stressed": 2})

In [ ]:
Non_stressed = bio_signals_train[bio_signals_train["labels"] == "Non_stressed"]
stressed  = bio_signals_train[bio_signals_train["labels"] == "Stressed"]
Neutral  = bio_signals_train[bio_signals_train["labels"] == "Neutral"]

In [ ]:

from matplotlib import pyplot as plt

from matplotlib.colors import ListedColormap

def bar_plot(ax, data, colors=None, total_width=0.8, single_width=1, legend=True):
    """Draws a bar plot with multiple bars per data point.

    Parameters
    ----------
    ax : matplotlib.pyplot.axis
        The axis we want to draw our plot on.

    data: dictionary
        A dictionary containing the data we want to plot. Keys are the names of the
        data, the items is a list of the values.

        Example:
        data = {
            "x":[1,2,3],
            "y":[1,2,3],
            "z":[1,2,3],
        }

    colors : array-like, optional
        A list of colors which are used for the bars. If None, the colors
        will be the standard matplotlib color cyle. (default: None)

    total_width : float, optional, default: 0.8
        The width of a bar group. 0.8 means that 80% of the x-axis is covered
        by bars and 20% will be spaces between the bars.

    single_width: float, optional, default: 1
        The relative width of a single bar within a group. 1 means the bars
        will touch eachother within a group, values less than 1 will make
        these bars thinner.

    legend: bool, optional, default: True
        If this is set to true, a legend will be added to the axis.
    """

    # Check if colors where provided, otherwhise use the default color cycle
    
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # Number of bars per group
    n_bars = len(data)

    # The width of a single bar
    bar_width = total_width / n_bars

    # List containing handles for the drawn bars, used for the legend
    bars = []

    # Iterate over all data
    for i, (name, values) in enumerate(data.items()):
        # The offset in x direction of that bar
        x_offset = (i - n_bars / 2) * bar_width + bar_width / 2

        # Draw a bar for every value of that type
        for x, y in enumerate(values):
            bar = ax.bar(x + x_offset, y, width=bar_width * single_width, color=colors[i % len(colors)])

        # Add a handle to the last drawn bar, which we'll need for the legend
        bars.append(bar[0])

    # Draw legend if we need
    if legend:
        ax.legend(bars, data.keys())


if __name__ == "__main__":
    # Usage example:
    N = 3
    ind = np.arange(N)  # the x locations for the groups
#     width = 0.08 
    data = {
        "Neutral": [4831, 17592, 2289],
        "Stressed": [2289, 17592, 2289],
        "Non-stressed": [17592, 17592, 2289],
    }
    fig, ax = plt.subplots()
    bar_plot(ax, data, total_width=.8, single_width=.9)
    ax.set_xticks(ind)
    ax.set_xticklabels( ('original Label distribution', 'After SMOTE', 'After re-sample') )
    plt.show()



### Overfitting the model with one participant

In [ ]:
bio_signals_validation=pd.DataFrame()
# for i in partition_devel_index:
bio_signals=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/20.csv")
target=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/20.csv")
#     fau=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/faus/{i}.csv")
#     fau.drop(['timestamp','subject_id'],axis=1,inplace=True)
bio_signals=bio_signals[['ECG','resp','BPM']]
#     valence=valence[['value']]
bio_signals['labels']=target['label']
bio_signals_validation = pd.concat([bio_signals_validation, bio_signals])
# bio_signals_validation['BPM'] = zscore(bio_signals_validation['BPM'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

clf = LogisticRegression(max_iter=1000)
clf.fit(x,y)

In [ ]:
y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

In [ ]:
plt.hist(bio_signals_train['labels'])

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x=bio_signals_train[bio_signals_train.columns[2:-1]]
y=bio_signals_train['labels'] 

# cross_val_score(XGBClassifier(), X, y)

clf = XGBClassifier()
clf.fit(x,y)

y_pred=clf.predict(bio_signals_validation[x.columns])
print(clf.score(bio_signals_validation[x.columns],bio_signals_validation['labels']))
print(classification_report(bio_signals_validation['labels'], y_pred))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf,bio_signals_validation[x.columns], bio_signals_validation['labels'])

## Normalizing the features with Minmax

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scaled = scaler.fit_transform(data)

In [ ]:
# for i in labels:
#     data=pd.read_csv(f"D:/Thesis_data/c3_muse_stress_2022/label_segments/target_labels/{i}.csv")
#     one_hot_encoded_data = pd.get_dummies(data, columns = ['labels'])
from sklearn.preprocessing import OneHotEncoder


import math
label=[]
label.extend(partition_train_index)
label.extend(partition_devel_index)
label.extend(partition_test_index)
for i in label:
    Biosignals=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/{i}.csv")

    target=pd.DataFrame()
    target=Biosignals[['timestamp','subject_id','ECG','resp','BPM']]
    target[['ECG','resp','BPM']] = scaler.fit_transform(Biosignals[['ECG','resp','BPM']])
#     target['timestamp']=Biosignals['timestamp']
    print(target)

  
    target.to_csv(f"D:/Thesis_data/c3_muse_stress_2022/feature_segments/Normalized_biosignals/{i}.csv",index=False)




## Valence and arousal distribution of the dataset

In [ ]:
train_arousal=[]
devel_arousal=[]
for i in partition_train_index:
    arousal=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/physio-arousal/{i}.csv")
    train_arousal.extend(arousal['value'])
for i in partition_devel_index:
    arousal=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/physio-arousal/{i}.csv")
    devel_arousal.extend(arousal['value'])



In [ ]:
sns.kdeplot(train_arousal, label='train')
sns.kdeplot(devel_arousal, label='devel')
print(np.mean(train_arousal))
print(np.mean(devel_arousal))

print(np.std(train_arousal))
print(np.std(devel_arousal))

# Set the title and axis labels
plt.title("Arousal value distribution")
plt.xlabel("Values")
plt.ylabel("Density")
plt.legend()
# Show the plot
plt.show()